In [1]:
%env CUDA_VISIBLE_DEVICES=0
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
import imageio
import moviepy.editor as mpy

BATCH_SIZE = 5000
SAMPLE_COL = 16
SAMPLE_ROW = 16
SAMPLE_NUM = SAMPLE_COL * SAMPLE_ROW
IMG_H = 28
IMG_W = 28
IMG_C = 1
IMG_SHAPE = (IMG_H, IMG_W, IMG_C)
Z_DIM = 128
BUF = 65536
DC_LR = 2.5e-04
DC_EPOCH = 256
W_LR = 2.0e-04
W_EPOCH = 256
BZ = (BATCH_SIZE, Z_DIM)
WClipLo = -0.01
WClipHi = 0.01

env: CUDA_VISIBLE_DEVICES=0


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit = 0x2400)])

In [7]:
(train_images, _), (test_images, _) = tf.keras.datasets.mnist.load_data()

iTrain = train_images.reshape(-1, 28, 28, 1).astype(np.float32)

# Normalizing the images to the range of [0., 1.]
iTrain = iTrain / 255.0
print(iTrain.shape)

dsTrain = tf.data.Dataset.from_tensor_slices(iTrain).shuffle(BUF).batch(BATCH_SIZE, True)
print(dsTrain)

# Utility function
def utPuzzle(imgs, row, col, path = None):
    h, w, c = imgs[0].shape
    out = np.zeros((h * row, w * col, c), np.uint8)
    for n, img in enumerate(imgs):
        j, i = divmod(n, col)
        out[j * h : (j + 1) * h, i * w : (i + 1) * w, :] = img
    if path is not None : imageio.imwrite(path, out)
    return out
  
def utMakeGif(imgs, fname, duration):
    n = float(len(imgs)) / duration
    clip = mpy.VideoClip(lambda t : imgs[int(n * t)], duration = duration)
    clip.write_gif(fname, fps = n)

(60000, 28, 28, 1)
<BatchDataset shapes: (5000, 28, 28, 1), types: tf.float32>


In [4]:
def GAN(sp, zd):
    # Shape
    xh, xw, xc = sp
    zh = xh // 4
    zw = xw // 4
        
    # Model
    return keras.Sequential([
        keras.layers.Dense(units  =  1024, input_shape = (zd,)),
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.Dense(units  =  zh * zw << 8),
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.Reshape(target_shape = (zh, zw, 256)),
        keras.layers.Conv2DTranspose(
            filters = 32,
            kernel_size = 5,
            strides = 2,
            padding = "SAME"
        ),
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.Conv2DTranspose(
            filters = xc,
            kernel_size = 5,
            strides = 2,
            padding = "SAME",
            activation = keras.activations.sigmoid
        ),
    ]), keras.Sequential([
        keras.layers.Conv2D(
            filters = 32,
            kernel_size = 5,
            strides = (2, 2),
            padding = "SAME",
            input_shape = sp,
        ),
        keras.layers.LeakyReLU(),
        #keras.layers.Dropout(0.3),
        keras.layers.Conv2D(
            filters = 128,
            kernel_size = 5,
            strides = (2, 2),
            padding = "SAME"
        ),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(),
        #keras.layers.Dropout(0.3),
        keras.layers.Flatten(),
        keras.layers.Dense(units  =  1024),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(),
        keras.layers.Dense(units  =  1),
    ])

s = tf.random.normal([SAMPLE_NUM, Z_DIM])

In [5]:
DCG, DCD = GAN(IMG_SHAPE, Z_DIM)
opg = keras.optimizers.Adam(DC_LR)
opd = keras.optimizers.Adam(DC_LR)

cross_entropy = keras.losses.BinaryCrossentropy(from_logits = True)

def DCGLoss(c0):
    return cross_entropy(tf.ones_like(c0), c0)

def DCDLoss(c0, c1):
    l1 = cross_entropy(tf.ones_like(c1), c1)
    l0 = cross_entropy(tf.zeros_like(c0), c0)
    return l1 + l0

@tf.function
def DCDTrain(c1):
    z = tf.random.normal(BZ)

    with tf.GradientTape() as tp:
        c0 = DCG(z, training = True)

        z1 = DCD(c1, training = True)
        z0 = DCD(c0, training = True)

        lg = DCGLoss(z0)
        ld = DCDLoss(z0, z1)

    gd = tp.gradient(ld, DCD.trainable_variables)

    opd.apply_gradients(zip(gd, DCD.trainable_variables))
    
    return lg, ld

@tf.function
def DCGTrain(c1):
    z = tf.random.normal(BZ)

    with tf.GradientTape() as tp:
        c0 = DCG(z, training = True)

       
        z1 = DCD(c1, training = True)
        z0 = DCD(c0, training = True)

        lg = DCGLoss(z0)
        ld = DCDLoss(z0, z1)

    gg = tp.gradient(lg, DCG.trainable_variables)

    opg.apply_gradients(zip(gg, DCG.trainable_variables))
    
    return lg, ld


DCTrain = (
    DCDTrain,
    DCDTrain,
    DCDTrain,
    DCDTrain,
    DCDTrain,
    DCGTrain
)

DCCritic = len(DCTrain)

In [5]:
print("Generator Initial Output :")
c0 = DCG(tf.random.normal((1, Z_DIM)), training = False)
plt.imshow((c0[0, :, :, 0] * 255.0).numpy().astype(np.uint8), cmap = "gray")
plt.axis("off")
plt.show()
print("Discriminator Initial Output : %E" % DCD(c0).numpy())

Generator Initial Output :


NameError: name 'DCG' is not defined

In [7]:
dclg = [None] * DC_EPOCH
dcld = [None] * DC_EPOCH
dcsp = [None] * DC_EPOCH
rsTrain = float(BATCH_SIZE) / float(len(iTrain))
ctr = 0
for ep in range(DC_EPOCH):
    lgt = 0.0
    ldt = 0.0
    for c1 in dsTrain:
        lg, ld = DCTrain[ctr](c1)
        ctr += 1
        lgt += lg.numpy()
        ldt += ld.numpy()
        if ctr == DCCritic : ctr = 0
    dclg[ep] = lgt * rsTrain
    dcld[ep] = ldt * rsTrain
    
    out = DCG(s, training = False)
    img = utPuzzle(
        (out * 255.0).numpy().astype(np.uint8),
        SAMPLE_COL,
        SAMPLE_ROW,
        "imgs/dc_%04d.png" % ep
    )
    dcsp[ep] = img
    if ep & 0x1F == 0x1F:
        plt.imshow(img[..., 0], cmap = "gray")
        plt.axis("off")
        plt.title("Epoch %d" % ep)
        plt.show()

FileNotFoundError: The directory '/home/mike/jupyter_notebook/Lab14-2/imgs' does not exist

In [5]:
WG, WD = GAN(IMG_SHAPE, Z_DIM)
opg = keras.optimizers.RMSprop(W_LR)
opd = keras.optimizers.RMSprop(W_LR)

@tf.function
def WGTrain(c1):
    z = tf.random.normal(BZ)

    with tf.GradientTape() as tpg:
        c0 = WG(z, training = True)

        z1 = WD(c1, training = True)
        z0 = WD(c0, training = True)

        ld = tf.reduce_mean(z0)
        lg = - ld
        ld = ld - tf.reduce_mean(z1)

    gg = tpg.gradient(lg, WG.trainable_variables)

    opg.apply_gradients(zip(gg, WG.trainable_variables))
    
    return lg, ld

@tf.function
def WDTrain(c1):
    z = tf.random.normal(BZ)

    with tf.GradientTape() as tpd:
        c0 = WG(z, training = True)

        z1 = WD(c1, training = True)
        z0 = WD(c0, training = True)

        ld = tf.reduce_mean(z0)
        lg = - ld
        ld = ld - tf.reduce_mean(z1)

    gd = tpd.gradient(ld, WD.trainable_variables)

    opd.apply_gradients(zip(gd, WD.trainable_variables))
    for v in WD.trainable_variables : v.assign(tf.clip_by_value(v, WClipLo, WClipHi))
    
    return lg, ld

WTrain = (
    WDTrain,
    WDTrain,
    WDTrain,
    WDTrain,
    WDTrain,
    WGTrain
)

WCritic = len(WTrain)

In [7]:
wlg = [None] * W_EPOCH
wld = [None] * W_EPOCH
wsp = [None] * W_EPOCH
rsTrain = float(BATCH_SIZE) / float(len(iTrain))
ctr = 0
for ep in range(W_EPOCH):
    lgt = 0.0
    ldt = 0.0
    for c1 in dsTrain:
        lg, ld = WTrain[ctr](c1)
        ctr += 1
        lgt += lg.numpy()
        ldt += ld.numpy()
        if ctr == WCritic : ctr = 0
    wlg[ep] = lgt * rsTrain
    wld[ep] = ldt * rsTrain
    
    out = WG(s, training = False)
    img = utPuzzle(
        (out * 255.0).numpy().astype(np.uint8),
        SAMPLE_COL,
        SAMPLE_ROW,
        "img_align_celeba_png/%06d.png" % ep
    )
    wsp[ep] = img
    if ep & 0x1F == 0x1F:
        
        plt.imshow(img[..., 0], cmap = "gray")
        plt.axis("off")
        plt.title("Epoch %d" % ep)
        plt.show()

KeyboardInterrupt: 

In [ ]:
utMakeGif(np.array(wsp), "imgs/wgan.gif", duration = 2)

In [ ]:
plt.plot(range(W_EPOCH), wld, color = "blue", label = "Discriminator Loss")
plt.plot(range(W_EPOCH), wlg, color = "red", label = "Generator Loss")
plt.legend(loc = "upper right")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("WGAN Training Loss")
plt.show()